In [422]:
library(readxl)
library(data.table)
library(ggplot2)
library(ggprism)
library(ggforce)
library(ggetho)
library(ggsignif)
library(zoo)
theme_set(theme_prism(base_size=20))
options(repr.plot.width=15, repr.plot.height=15)
source("R/plot.R")
source("R/Cbind.R")
source("R/link_idoc_metadata.R")
source("R/library.R")

This notebook reads the IDOC results stored in "IDOC_METHODS_PAPER_METADATA.xlsx" and produces a csv file with the data in tidy format

* Fly_group_experiment_treatment: experimental information of the fly 
* fly_name_reference: unique animal identifier, built as follows: date_time(scientist_suffix)_ROI_X
* experiment: type of memory evaluated (20min STM, 1h STM, LTM, LTM CHX, ...)
* id
* test: (PI is read PRE or POST conditioning)
* PI: (value of PI)
* Training_session: how many times CS-US pairing is presented
* Genotype: genotype of the fly
* interval: NONE, or ZTX-ZTY if the fly was SD'd between ZT X and ZT Y
* ZT4,ZT4.5,ZT5,ZT5.5,ZT6,ZT6.5,ZT7,ZT7.5,ZT8,ZT8.5,ZT9,ZT9.5,ZT10,ZT10.5,ZT11,ZT11.5,ZT12,ZT12.5,ZT13,ZT13.5,ZT14,ZT14.5,ZT15,ZT15.5,ZT16,ZT16.5,ZT17,ZT17.5,ZT18,ZT18.5,ZT19,ZT19.5,ZT20,ZT20.5,ZT21,ZT21.5,ZT22,ZT22.5,ZT23,ZT23.5,ZT24,ZT24.5,ZT25,ZT25.5,ZT26,ZT26.5,ZT27,ZT27.5,ZT28,ZT28.5,ZT29,ZT29.5,ZT30
* ZT4_interactions,ZT4.5_interactions,ZT5_interactions,ZT5.5_interactions,ZT6_interactions,ZT6.5_interactions,ZT7_interactions,ZT7.5_interactions,ZT8_interactions,ZT8.5_interactions,ZT9_interactions,ZT9.5_interactions,ZT10_interactions,ZT10.5_interactions,ZT11_interactions,ZT11.5_interactions,ZT12_interactions,ZT12.5_interactions,ZT13_interactions,ZT13.5_interactions,ZT14_interactions,ZT14.5_interactions,ZT15_interactions,ZT15.5_interactions,ZT16_interactions,ZT16.5_interactions,ZT17_interactions,ZT17.5_interactions,ZT18_interactions,ZT18.5_interactions,ZT19_interactions,ZT19.5_interactions,ZT20_interactions,ZT20.5_interactions,ZT21_interactions,ZT21.5_interactions,ZT22_interactions,ZT22.5_interactions,ZT23_interactions,ZT23.5_interactions,ZT24_interactions,ZT24.5_interactions,ZT25_interactions,ZT25.5_interactions,ZT26_interactions,ZT26.5_interactions,ZT27_interactions,ZT27.5_interactions,ZT28_interactions,ZT28.5_interactions,ZT29_interactions,ZT29.5_interactions,ZT30_interactions
User


In [423]:
database <- "IDOC_metadata_AOJ - metadata.csv"
metadata_aoj <- data.table::fread(database)[fly_name_reference != "" & comment=="OK",]
metadata_aoj<-metadata_aoj[keep==TRUE,]
metadata_aoj<-metadata_aoj[Training%in%c("1X", "6X_Spaced", "6X_Massed"),]

metadata_etho_aoj <- data.table::fread("IDOC_metadata_AOJ - ethoscope_LTM.csv")
metadata_aoj <- merge(
    metadata_aoj,
    metadata_etho_aoj[, .(ROI, Files, region_id, date, machine_name, schedule, SD_quality, SD_quality_second_eye, interactor, interactor_time_window)],
    by=c("ROI", "Files"),
    all=TRUE
)
# # this information is to be retrieved from a merge with the ethoscope metadata .csv
# metadata_aoj$interactor<-"NONE"
# metadata_aoj$interactor_time_window <-"NONE"
# metadata_aoj$SD_quality<-"NONE"
# metadata_aoj$SD_quality_second_eye <-"NONE"
# metadata_aoj$region_id <-"NONE"
# metadata_aoj$machine_name <-"NONE"
# metadata_aoj$date <-"NONE"

In [425]:
database <- "/idoc_data/paper/CHuiData.ods"
sheets <- readODS::list_ods_sheets(database)
columns <- c(
    "Files", "Genotype", "ROI", "Gender", "Age", "ATR", "Training",
    "Odor", "Sugar", "Electric shock",  "PRE", "PRE_1", "PRE_2", "POST", "POST_1", "POST_2", "Keep",
    "fly_name_reference", "experiment"
)
metadata_ch <- read_idoc_metadata(file = database, sheets=sheets, columns = columns, backend=readODS::read_ods)
metadata_ch$SD_quality <- "NONE"
metadata_ch$SD_quality_second_eye <- "NONE"
metadata_ch$interactor_time_window <- "No_stimulator"
metadata_ch$interactor <- "DefaultStimulator"
metadata_ch$`comment on SD` <- NA
metadata_ch$User <- "CH"
metadata_ch$region_id <- "NONE"

In [426]:
# metadata_ch$POST

In [427]:
database <- "idoc_metadata.ods"
sheets <- c("20min_STM", "1hr_STM", "3hr_STM")
columns <- c(
    "Files", "Genotype", "ROI", "User", "fly_name_reference", "Gender", "Age", "ATR", "Training",
    "Odor", "Sugar", "Electric shock",  "PRE", "PRE_1", "PRE_2", "POST", "POST_1", "POST_2", "Keep",
    "SD_quality", "SD_quality_second_eye", "interactor_time_window", "interactor", "comment on SD", "experiment"
)

## Read STM experiments

In [428]:
metadata_stm <- read_idoc_metadata(file = database, sheets=sheets, columns = columns, backend=readODS::read_ods)
# STM flies don't go to the ethoscope, so region_id is NONE
metadata_stm$region_id <- "NONE"

Warning message in doTryCatch(return(expr), name, parentenv, handler):
“Deleting 21 42 63 82 103 124 145 166 rows from metadata”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“Deleting 21 42 63 rows from metadata”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“Deleting 21 42 63 84 105 126 rows from metadata”


In [429]:
metadata_ltm <- read_idoc_metadata(file = database, sheets=c("24hr_LTM"), columns = c(columns, "date", "machine_name", "region_id"), backend=readODS::read_ods)
# keep either NS or good SD flies, (discard bad SD flies) 
metadata_ltm <- metadata_ltm[SD_quality %in% c("No_sleep_depriver", "good") & SD_quality_second_eye %in% c("No_sleep_depriver", "good"),]
metadata_ltm[SD_quality=="No_sleep_depriver", SD_quality := "NONE"]
metadata_ltm[SD_quality_second_eye=="No_sleep_depriver", SD_quality_second_eye := "NONE"]


Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“Deleting 20 38 58 79 100 117 128 139 150 161 172 183 194 205 216 227 238 249 259 270 281 292 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 357 368 379 390 401 rows from metadata”


In [430]:
table(metadata_ltm$interactor_time_window)


No_stimulator     ZT05-ZT11     ZT05-ZT24   ZT12-ZT14.5     ZT12-ZT18 
          122            56            28             9            61 

In [431]:
metadata_gtacr <- read_idoc_metadata(file = database, sheets=c("24hr_LTM_GTACR"), columns =  c(columns, "date", "machine_name", "region_id"), backend=readODS::read_ods)
metadata_gtacr[SD_quality=="No_sleep_depriver", SD_quality := "NONE"]
metadata_gtacr[SD_quality_second_eye=="No_sleep_depriver", SD_quality_second_eye := "NONE"]


Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“Deleting 11 22 33 44 55 66 rows from metadata”


In [432]:
metadata <- rbind(
    metadata_stm[, .(fly_name_reference, date=NA_character_, reference_hour=NA_real_, machine_name=NA_character_, region_id=NA, Files, ROI, Genotype, Gender, Age, interactor, interactor_time_window, Training, SD_quality, SD_quality_second_eye, `comment on SD`, User, experiment, PRE_1, PRE_2, PRE, POST_1, POST_2, POST)],
    metadata_ltm[, .(fly_name_reference, date=as.character(date), reference_hour=11, machine_name, region_id, Files, ROI, Genotype, Gender, Age, interactor, interactor_time_window, Training, SD_quality, SD_quality_second_eye, `comment on SD`, User, experiment, PRE_1, PRE_2, PRE, POST_1, POST_2, POST)],
    metadata_gtacr[, .(fly_name_reference, date=as.character(date), reference_hour=NA_real_, machine_name, region_id, Files, ROI, Genotype, Gender, Age, interactor, interactor_time_window, Training, SD_quality, SD_quality_second_eye, `comment on SD`, User, experiment, PRE_1, PRE_2, PRE, POST_1, POST_2, POST)],
    metadata_ch[, .(fly_name_reference, date=NA_character_, reference_hour=11, machine_name=NA_character_, region_id=NA, Files, ROI, Genotype, Gender, Age, interactor, interactor_time_window, Training, SD_quality, SD_quality_second_eye, `comment on SD`, User, experiment, PRE_1, PRE_2, PRE, POST_1, POST_2, POST)],
    metadata_aoj[, .(fly_name_reference, date=as.character(date), reference_hour, machine_name, region_id, Files, ROI, Genotype, Gender, Age, interactor, interactor_time_window, Training, SD_quality, SD_quality_second_eye, `comment on SD`="NONE", User, experiment, PRE_1, PRE_2, PRE, POST_1, POST_2, POST)]
)

In [433]:
data.table::fwrite(x = metadata, file="metadata.csv")

In [434]:
metadata_etho <- read_idoc_metadata(file = database, sheets=c("24hr_LTM"), columns = c(columns, "date", "machine_name", "region_id"), backend=readODS::read_ods)[is.na(ROI),]

Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“NAs introduced by coercion”
Warning message in doTryCatch(return(expr), name, parentenv, handler):
“Deleting 20 38 58 79 100 117 128 139 150 161 172 183 194 205 216 227 238 249 259 270 281 292 303 304 305 306 307 308 309 310 311 312 313 314 315 316 317 318 319 320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335 336 337 338 339 340 341 342 343 344 345 346 357 368 379 390 401 rows from metadata”


In [435]:
metadata_etho[, reference_hour := 11]
metadata_etho[, Genotype := "Iso31"]
metadata_etho$Files

[1] "2023-07-12_14-25-09_e36_roi_11-IDOC_roi_NA"
 [2] "2023-07-12_14-25-09_e36_roi_12-IDOC_roi_NA"
 [3] "2023-07-12_14-25-09_e36_roi_13-IDOC_roi_NA"
 [4] "2023-07-12_14-25-09_e36_roi_14-IDOC_roi_NA"
 [5] "2023-07-12_14-25-09_e36_roi_15-IDOC_roi_NA"
 [6] "2023-07-12_14-25-09_e36_roi_16-IDOC_roi_NA"
 [7] "2023-07-12_14-25-09_e36_roi_17-IDOC_roi_NA"
 [8] "2023-07-12_14-25-09_e36_roi_18-IDOC_roi_NA"
 [9] "2023-07-12_14-25-09_e36_roi_19-IDOC_roi_NA"
[10] "2023-07-13_16-05-32_e36_roi_11-IDOC_roi_NA"
[11] "2023-07-13_16-05-32_e36_roi_12-IDOC_roi_NA"
[12] "2023-07-13_16-05-32_e36_roi_13-IDOC_roi_NA"
[13] "2023-07-13_16-05-32_e36_roi_14-IDOC_roi_NA"
[14] "2023-07-13_16-05-32_e36_roi_15-IDOC_roi_NA"
[15] "2023-07-13_16-05-32_e36_roi_16-IDOC_roi_NA"
[16] "2023-07-13_16-05-32_e36_roi_17-IDOC_roi_NA"
[17] "2023-07-18_15-25-46_e42_roi_11-IDOC_roi_NA"
[18] "2023-07-18_15-25-46_e42_roi_12-IDOC_roi_NA"
[19] "2023-07-18_15-25-46_e42_roi_13-IDOC_roi_NA"
[20] "2023-07-18_15-25-46_e42_roi_14-IDOC_roi_NA"
[21] "2023-07-18_15-25-46_e42_roi_15-IDOC_roi_NA"
[22] "2023-07-18_15-25-46_e42_roi_16-IDOC_roi_NA"
[23] "2023-07-18_15-25-46_e42_roi_17-IDOC_roi_NA"
[24] "2023-07-18_15-25-46_e42_roi_18-IDOC_roi_NA"
[25] "2023-07-18_15-25-46_e42_roi_19-IDOC_roi_NA"
[26] "2023-07-31_15-16-30_e36_roi_11-IDOC_roi_NA"
[27] "2023-07-31_15-16-30_e36_roi_13-IDOC_roi_NA"
[28] "2023-07-31_15-16-30_e36_roi_15-IDOC_roi_NA"
[29] "2023-07-31_15-16-30_e36_roi_17-IDOC_roi_NA"
[30] "2023-07-31_15-16-30_e36_roi_19-IDOC_roi_NA"
[31] "2023-07-31_15-16-30_e36_roi_20-IDOC_roi_NA"

In [436]:
data.table::fwrite(x = metadata_etho, file="metadata_etho.csv")